In [1]:
%matplotlib inline
import os
import sys
import logging
logging.basicConfig(format='%(asctime)s %(message)s', stream=sys.stdout)

import torch as T
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import numpy as np
from tqdm import tqdm_notebook as tqdm

from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

In [2]:
def read_dataset(model: str, task: str, split: str,  basepath='/scratch/fl1092/ml_protein_data'):
    path_to_file = f'{basepath}/{model}/{task}/{task}_{split}.p'
    data = np.load(path_to_file, allow_pickle=True)
    
    return data

def dict_2_arr(data_dict, labels, avgr=lambda x: np.mean(x, axis=0)):
    
    emb_shape = list(data_dict.values())[0].shape
    number_of_embeddings = len(data_dict) 

    X = np.zeros((number_of_embeddings, emb_shape[-1]))
    y = np.zeros(number_of_embeddings)
    
    i = 0

    # iter over sorted keys in labels to ensure proteins
    # from different models are indexed the same
    keys = list(labels.keys())
    keys.sort()
    for key in keys :
        if key == 'd1smyc_':
            continue
        X[i] = avgr(data_dict[key])
        y[i] = labels[key]
        i += 1
        
    return X, y

def ensemble_append_mean_reps(dicts, labels):
    
    new_dict = dict()
    keys = dicts[0].keys()
    for key in keys:
        seqs = []
        for d in dicts:
            seq = np.mean(d[key], axis=0)
            seq = preprocessing.normalize([seq], norm='l2')
            seqs.append(seq)
        combined_seqs = np.concatenate(seqs, axis=1)
        new_dict[key] = combined_seqs

    emb_size = list(new_dict.values())[0].shape[1]
    X = np.zeros((len(new_dict), emb_size))
    y = np.zeros(len(new_dict))
    
    i = 0
    for key in new_dict:
        X[i] = new_dict[key]
        y[i] = labels[key]
        i += 1
        
    return X, y

In [3]:
class DataSet(T.utils.data.Dataset):
    ### a Map-style dataset ###
    
    def __init__(self, task, split, device):
        self.elmo_train = read_dataset('elmo', task, split)
        self.unirep_train = read_dataset('unirep', task, split)
        self.y_data = read_dataset('label', task, split)
        
        X_app, self.y_data = ensemble_append_mean_reps([self.elmo_train, self.unirep_train], self.y_data)
        self.x_data = T.tensor(X_app, dtype=T.float32).to(device)
        
    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        if T.is_tensor(idx):
            idx = idx.tolist()
        
        x = self.x_data[idx, ]
        y = self.y_data[idx]
        
        return x, y

In [4]:
### encoder and decoder with two convolutional layers ###

class ConvEncoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        c = capacity
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=c, kernel_size=4, stride=2, padding=1) # out: c x 14 x 14
        self.conv2 = nn.Conv1d(in_channels=c, out_channels=c*2, kernel_size=4, stride=2, padding=1) # out: c x 7 x 7
        self.fc = nn.Linear(in_features=c*2*7*7, out_features=latent_dims)
            
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1) # flatten batch of multi-channel feature maps to a batch of feature vectors
        x = self.fc(x)
        return x

class ConvDecoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        c = capacity
        self.fc = nn.Linear(in_features=latent_dims, out_features=c*2*7*7)
        self.conv2 = nn.ConvTranspose1d(in_channels=c*2, out_channels=c, kernel_size=4, stride=2, padding=1)
        self.conv1 = nn.ConvTranspose1d(in_channels=c, out_channels=1, kernel_size=4, stride=2, padding=1)
            
    def forward(self, x):
        x = self.fc(x)
        # unflatten batch of feature vectors to a batch of multi-channel feature maps
        x = x.view(x.size(0), capacity*2, 7, 7)
        x = F.relu(self.conv2(x))
        # last layer before output is tanh, since the images are normalized and 0-centered
        x = torch.tanh(self.conv1(x))
        return x

In [5]:
### encoder and decoder with fully connected layers only ###

class Encoder(nn.Module):
    def __init__(self, input_shape):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(in_features=input_shape, out_features=600)
        self.fc2 = nn.Linear(in_features=1024, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=128)
            
    def forward(self, x):
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = self.fc3(x)
        return x

class Decoder(nn.Module):
    def __init__(self, input_shape):
        super(Decoder, self).__init__()
        #self.fc1 = nn.Linear(in_features=128, out_features=128)
        #self.fc2 = nn.Linear(in_features=128, out_features=1024)
        self.fc3 = nn.Linear(in_features=600, out_features=input_shape)
            
    def forward(self, x):
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

In [6]:
class Autoencoder(nn.Module):
    def __init__(self, input_shape):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(input_shape)
        self.decoder = Decoder(input_shape)
    
    def forward(self, x):
        latent = self.encoder(x)
        x_recon = self.decoder(latent)
        return x_recon

In [7]:
### loss functions ###

def caemeLoss(outputs, batch_features, latent, d1, d2, l1=1, l2=1, l3=1):
    mse = nn.MSELoss()
    o1 = outputs[:, 0:d1]
    o2 = outputs[:, d1:d1+d2]
    f1 = batch_features[:, 0:d1]
    f2 = batch_features[:, d1:d1+d2] 
    
    return l1*mse(o1, f1) + l2*mse(o2, f2)

def mseLoss(outputs, batch_features, latent, d1, d2, l1=1, l2=1, l3=1):
    # mean-squared error loss
    mse = nn.MSELoss() 
    
    return mse(outputs, batch_features)

In [12]:
### pipeline for training meta embedding and classifying ###
def metaEmbedding(data, model):
    X_latent = []
    Y = []

    for i in range(len(data)):
        x, y = data[i]
        X_latent.append(model.encoder(x).cpu().detach().numpy())
        Y.append(y)
        
    return X_latent, Y

def autoencoderPipeline(task, num_epochs=30, learning_rate=1e-3, batch_size=128, loss_function=mseLoss):
    device = T.device('cuda')
    
    print('Loading data ...')
    train_data = DataSet(task, 'train', device)
    test_data = DataSet('remote_homology', 'test', device)
    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    
    model = Autoencoder(2924).to(device)
    optimizer = T.optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay=1e-7)
    
    print('Training autoencoder...')
    for epoch in tqdm(range(num_epochs)):
        loss = 0
        for batch_features, _ in train_dataloader:
            
            batch_features = batch_features.to(device) # load data to GPU
            optimizer.zero_grad() # reset the gradients back to zero
            
            outputs = model(batch_features) # compute reconstructions
            latent = model.encoder(batch_features) # compute latent meta embedding
            
            train_loss = loss_function(outputs, batch_features, latent, 1024, 1900) # compute training loss
            train_loss.backward() # compute accumulated gradients
            optimizer.step() # perform parameter update based on current gradients
            loss += train_loss.item() # add the mini-batch training loss to epoch loss

        loss = loss / len(train_dataloader) # compute the epoch training loss

        if epoch %3 == 0:
            print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, num_epochs, loss))
            
    print('Training classifier ...')
    
    test_x_latent, test_y = metaEmbedding(test_data, model)
    train_x_latent, train_y = metaEmbedding(train_data, model)
    
    clf = OneVsRestClassifier(LinearSVC())
    clf.fit(train_x_latent, train_y)
    score = clf.score(test_x_latent, test_y)
    
    print(f"Score on test set: {score}")

# Experiments

In [13]:
%%time
autoencoderPipeline('remote_homology', loss_function=mseLoss)

Loading data ...
Training autoencoder...


epoch : 1/30, loss = 0.000525
epoch : 4/30, loss = 0.000464
epoch : 7/30, loss = 0.000450
epoch : 10/30, loss = 0.000443
epoch : 13/30, loss = 0.000440
epoch : 16/30, loss = 0.000439
epoch : 19/30, loss = 0.000438
epoch : 22/30, loss = 0.000437
epoch : 25/30, loss = 0.000437
epoch : 28/30, loss = 0.000437

Training classifier ...
Score on test set: 0.25487465181058494
CPU times: user 3min 50s, sys: 35.7 s, total: 4min 26s
Wall time: 4min 25s


In [14]:
%%time
autoencoderPipeline('remote_homology', loss_function=caemeLoss)

Loading data ...
Training autoencoder...


epoch : 1/30, loss = 0.001178
epoch : 4/30, loss = 0.001067
epoch : 7/30, loss = 0.001046
epoch : 10/30, loss = 0.001031
epoch : 13/30, loss = 0.001025
epoch : 16/30, loss = 0.001021
epoch : 19/30, loss = 0.001018
epoch : 22/30, loss = 0.001015
epoch : 25/30, loss = 0.001014
epoch : 28/30, loss = 0.001013

Training classifier ...
Score on test set: 0.2646239554317549
CPU times: user 3min 49s, sys: 36 s, total: 4min 25s
Wall time: 4min 24s
